## Importing Useful Libraries

In [ ]:
from collections import defaultdict
import pandas as pd
import numpy as np
import yfinance as yf
import warnings
import datetime as dt
import scipy.stats as stats
warnings.filterwarnings("ignore")

!pip install py_vollib
from py_vollib.black_scholes.implied_volatility import implied_volatility as iv
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import vega

## Data

In [2]:
def get_strike_price_pivot_table(ticker, maturity_min=0.07, maturity_max=1, moneyness_min=0.95, moneyness_max=1.2, option_type="call"):

    '''
    Generate a pivot table of option strike prices for a given ticker.
    This function fetches option data for a specified stock ticker and creates a pivot table of strike prices across different maturities. It filters the data based on time to maturity and moneyness (strike price relative to spot price).
    Args:
    ticker (str): The stock ticker symbol (e.g., AAPL for Apple Inc.). maturity_min (float, optional): Minimum time to maturity in years. Defaults to 0.1.

    maturity max (float, optional): Maximum time to maturity in years. Defaults to 2. moneyness min (float, optional): Minimum moneyness (strike/spot) to consider. Defaults to 0.95. moneyness max (float, optional): Maximum moneyness (strike/spot) to consider. Defaults to 1.2. option type (str, optional): The type of option (call or put'). Defaults to 'call'.

    Returns:

    pd.DataFrame: A pivot table with time to maturity as index, strike prices as columns, and option last prices as values. Strikes not available for a particular maturity are filled with 0.
    '''
    if option_type not in ["call", "put"]:

        raise ValueError("Invalid option type. Choose 'call' or 'put'.")

    option_data = yf.Ticker(ticker)

    spot_price= option_data.history("1d")["Close"].iloc[0]

    today = pd. Timestamp.today()

    valid_maturities = [

        mat for mat in option_data.options

        if maturity_min < (pd.to_datetime(mat) - today).days / 365 < maturity_max
    ]

    strikes_freq = defaultdict(int)

    all_data = []

    for maturity in valid_maturities:
        if option_type == "call":
            chain = option_data.option_chain(maturity).calls
        elif option_type == "put":
            chain = option_data.option_chain(maturity).puts

        ttm = (pd.to_datetime(maturity) - today).days / 365

        valid_strikes = chain[

            (chain ["strike"] > moneyness_min * spot_price) &

            (chain["strike"] <= moneyness_max * spot_price)

        ]

        for strike in valid_strikes["strike"]:
            strikes_freq[strike] += 1

        valid_strikes["TTM"] = ttm
        all_data.append(valid_strikes[["strike", "lastPrice", "TTM"]])

    common_strikes = {strike for strike, freq in strikes_freq.items() if freq == len(valid_maturities)}

    combined_data = pd.concat(all_data, ignore_index=True)
    combined_data = combined_data[combined_data["strike"].isin(common_strikes)]

    pivot_table = combined_data.pivot_table(index="TTM", columns="strike", values="lastPrice",fill_value=0)

    return pivot_table

In [3]:
data = yf.download('AAPL',period='1y')
a = get_strike_price_pivot_table('AAPL')
a.reset_index(inplace=True)
a

[*********************100%%**********************]  1 of 1 completed


strike,TTM,220.0,225.0,230.0,235.0,240.0,245.0,250.0,260.0,270.0
0,0.082192,12.15,8.95,6.25,4.27,2.79,1.92,1.15,0.53,0.24
1,0.101370,12.45,9.35,6.80,4.75,3.24,2.15,1.41,0.61,0.30
2,0.120548,13.15,10.10,7.30,5.18,3.75,2.48,1.75,0.79,0.38
3,0.197260,15.13,12.10,9.35,7.13,5.35,4.01,2.90,1.58,0.83
4,0.273973,17.10,13.90,11.30,9.00,6.82,5.40,4.25,2.45,1.41
5,0.350685,19.50,16.40,13.78,11.42,9.32,7.64,6.15,4.10,2.54
6,0.446575,21.30,18.35,15.55,13.15,11.05,9.10,7.60,5.00,3.40
7,0.523288,22.59,19.70,17.00,14.60,12.18,10.30,8.75,5.95,4.16
8,0.695890,26.05,23.10,20.55,17.70,15.46,13.49,11.80,8.70,6.60
9,0.945205,30.85,27.93,25.28,22.49,20.15,17.90,16.05,12.80,9.90


## Function to calculate Nearest OTM (or ATM if it exists) to Closing Price.

In [4]:
def calculate_otm(list,target):
    return min(list,key=lambda x: abs(x-target))

otm = calculate_otm(a.columns[1:-1],data['Close'].iloc[-1])
print(otm)

230.0


In [5]:
last_option_prices=a[otm]
ttm=a['TTM']
print(last_option_prices)
# print(ttm)

0     6.25
1     6.80
2     7.30
3     9.35
4    11.30
5    13.78
6    15.55
7    17.00
8    20.55
9    25.28
Name: 230.0, dtype: float64


## Function to calculate Implied Volatility(IV)

In [6]:
from py_vollib.black_scholes.implied_volatility import implied_volatility as iv
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import vega
def implied_vol(S0, K, T, market_price, r=0.02 , flag='c', tol=0.00001):
    """Calculating the implied volatility of an European option
        S0: stock price
        K: strike price
        T: time to maturity
        r: risk-free rate
        market_price: option price in market
    """
    max_iter = 500 #max no. of iterations
    vol_old = 0.2 #initial guess
    for k in range(max_iter):
        bs_price = bs(flag, S0, K, T, r, vol_old)
        Cprime = vega(flag, S0, K, T, r, vol_old)*100
        C = bs_price - market_price
        vol_new = vol_old - C/Cprime
        new_bs_price = bs(flag, S0, K, T, r, vol_new)
        if (abs(vol_old-vol_new) < tol or abs(new_bs_price-market_price) < tol):
            break
        vol_old = vol_new
    implied_vol = vol_new
    return implied_vol

In [7]:
IV = {}

for i in range(len(last_option_prices)):

  iv = implied_vol(data['Close'].iloc[-1],otm,ttm[i],last_option_prices[i],0.02)
  IV[ttm[i]] = (iv).round(3)

IV2 = IV.copy()
print("Implied Volatility corresponding to different maturity:-",IV)

Implied Volatility corresponding to different maturity:- {0.0821917808219178: 0.272, 0.10136986301369863: 0.263, 0.12054794520547946: 0.255, 0.19726027397260273: 0.247, 0.273972602739726: 0.247, 0.3506849315068493: 0.261, 0.4465753424657534: 0.257, 0.5232876712328767: 0.257, 0.6958904109589041: 0.266, 0.9452054794520548: 0.276}


## Function to calculate Forward Volatility(FV)

In [8]:
FV = IV.copy()

for i in range(len(FV)):

  #Calculating annualized variance
  FV[ttm[i]]=(FV[ttm[i]] * FV[ttm[i]]).round(4)

  #Calculating one day variance
  FV[ttm[i]]=(FV[ttm[i]]/365)

  #Calculating 30-day variance
  FV[ttm[i]]=(FV[ttm[i]]*30).round(5)

for i in range(1,len(FV)):

  #Calculating 30-day forward variance
  FV[ttm[i]]=(FV[ttm[i]]-FV[ttm[i-1]])

for i in range(len(FV)):

    #Converting 30-day forward variance to 1-day variance
    FV[ttm[i]]=FV[ttm[i]]/30

    #Annualize forward variance
    FV[ttm[i]]=FV[ttm[i]]*365

    #Forward Volatility
    FV[ttm[i]]=((FV[ttm[i]]**(0.5))*100).round(3)

IV2=IV.copy()
for i in range(len(IV2)):
  IV2[ttm[i]]=(IV2[ttm[i]]*100).round(3)

print("Implied Volatility corresponding to different maturity:-",IV2)
print("Forward Volatility corresponding to different maturity:-",FV)

Implied Volatility corresponding to different maturity:- {0.0821917808219178: 27.2, 0.10136986301369863: 26.3, 0.12054794520547946: 25.5, 0.19726027397260273: 24.7, 0.273972602739726: 24.7, 0.3506849315068493: 26.1, 0.4465753424657534: 25.7, 0.5232876712328767: 25.7, 0.6958904109589041: 26.6, 0.9452054794520548: 27.6}
Forward Volatility corresponding to different maturity:- {0.0821917808219178: 27.198, 0.10136986301369863: nan, 0.12054794520547946: 26.404, 0.19726027397260273: nan, 0.273972602739726: 26.404, 0.3506849315068493: nan, 0.4465753424657534: 25.986, 0.5232876712328767: nan, 0.6958904109589041: 26.906, 0.9452054794520548: 6.141}


## Function to calculate Implied Volatility from Scratch using Newton-Raphson Method

In [9]:
import scipy.stats as stats

def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * stats.norm.cdf(d1) - K * np.exp(-r * T) * stats.norm.cdf(d2)
    return call_price

def vega(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return S * stats.norm.pdf(d1) * np.sqrt(T)

def implied_volatility_call(S, K, T, r, market_price, tol=1e-5, max_iterations=100):
    sigma = 0.2  # initial guess
    for i in range(max_iterations):
        price = black_scholes_call(S, K, T, r, sigma)
        vega_value = vega(S, K, T, r, sigma)
        price_diff = price - market_price

        if abs(price_diff) < tol:
            return sigma

        sigma = sigma - price_diff / vega_value

    raise ValueError("Implied volatility did not converge")

# Example data
S = data['Close'].iloc[-1]  # current stock price
K = otm  # strike price
T = ttm[0]  # time to expiry in years
r = 0.02  # risk-free rate
market_price = last_option_prices[0]  # market price of the call option

iv = implied_volatility_call(S, K, T, r, market_price)
print(f"Implied Volatility: {iv:.2%}")

Implied Volatility: 27.20%


In [ ]:
def get_strike_price_pivot_table(ticker,option_type, maturity_min=0.08, maturity_max=1, moneyness_min=0.6, moneyness_max=1.4):
    '''
    Generate a pivot table of option strike prices for a given ticker.
    This function fetches option data for a specified stock ticker and creates a pivot table of strike prices across different maturities. It filters the data based on time to maturity and moneyness (strike price relative to spot price).
    Args:
    ticker (str): The stock ticker symbol (e.g., AAPL for Apple Inc.). maturity_min (float, optional): Minimum time to maturity in years. Defaults to 0.1.

    maturity max (float, optional): Maximum time to maturity in years. Defaults to 2. moneyness min (float, optional): Minimum moneyness (strike/spot) to consider. Defaults to 0.95. moneyness max (float, optional): Maximum moneyness (strike/spot) to consider. Defaults to 1.2. option type (str, optional): The type of option (call or put'). Defaults to 'call'.

    Returns:

    pd.DataFrame: A pivot table with time to maturity as index, strike prices as columns, and option last prices as values. Strikes not available for a particular maturity are filled with 0.
    '''
    if option_type not in ["call", "put"]:
        raise ValueError("Invalid option type. Choose 'call' or 'put'.")
    option_data = yf.Ticker(ticker)
    spot_price= option_data.history("1d")["Close"].iloc[0]
    today = pd. Timestamp.today()
    valid_maturities = [
        mat for mat in option_data.options
        if maturity_min < (pd.to_datetime(mat) - today).days / 365 < maturity_max
    ]
    strikes_freq = defaultdict(int)
    all_data = []
    for maturity in valid_maturities:
        if option_type == "call":
            chain = option_data.option_chain(maturity).calls
        elif option_type == "put":
            chain = option_data.option_chain(maturity).puts
        ttm = (pd.to_datetime(maturity) - today).days / 365
        valid_strikes = chain[
            (chain ["strike"] > moneyness_min * spot_price) &

            (chain["strike"] <= moneyness_max * spot_price)
        ]
        for strike in valid_strikes["strike"]:
            strikes_freq[strike] += 1
        valid_strikes["TTM"] = ttm
        all_data.append(valid_strikes[["strike", "lastPrice", "TTM"]])
    common_strikes = {strike for strike, freq in strikes_freq.items() if freq == len(valid_maturities)}
    combined_data = pd.concat(all_data, ignore_index=True)
    combined_data = combined_data[combined_data["strike"].isin(common_strikes)]
    pivot_table = combined_data.pivot_table(index="TTM", columns="strike", values="lastPrice",fill_value=0)
    return pivot_table, spot_price

In [ ]:
ticker = 'NVDA'

call_data, spot_price = get_strike_price_pivot_table(ticker, 'call')
call_data.reset_index(inplace=True)
put_data, _ = get_strike_price_pivot_table(ticker, 'put')
put_data.reset_index(inplace=True)

call_data
#print(f'Last closing price of {ticker} is : {spot_price}')

strike,TTM,90.0,100.0,105.0,110.0,112.0,114.0,115.0,118.0,120.0,...,140.0,145.0,150.0,152.0,155.0,160.0,165.0,170.0,175.0,180.0
0,0.082192,38.24,31.70,24.95,22.83,20.35,19.00,18.03,16.10,14.42,...,4.15,2.84,1.90,1.69,1.30,0.87,0.57,0.42,0.29,0.23
1,0.101370,42.33,32.44,27.80,23.00,21.20,19.55,18.75,16.80,15.30,...,5.10,3.70,2.63,2.31,1.85,1.32,0.93,0.71,0.50,0.37
2,0.120548,43.75,31.06,27.75,24.10,24.80,21.16,20.42,17.30,17.15,...,7.27,5.63,4.56,4.08,3.50,2.68,2.12,1.57,1.25,0.99
3,0.197260,42.20,34.35,29.76,26.10,24.05,22.60,22.65,20.76,19.55,...,9.75,8.03,6.65,6.15,5.40,4.45,3.62,2.97,2.38,1.98
4,0.273973,43.04,35.35,30.90,27.36,25.95,25.00,23.80,22.27,21.25,...,11.66,9.70,8.40,7.70,7.20,5.90,5.00,4.17,3.55,2.98
5,0.350685,45.00,36.20,32.75,29.20,29.75,24.80,25.76,23.10,22.90,...,13.60,11.80,10.25,9.50,8.50,7.65,6.55,5.65,4.76,4.15
6,0.446575,45.83,38.40,34.55,31.47,29.85,28.46,27.93,26.15,25.35,...,15.95,14.20,12.60,12.07,11.15,9.96,8.60,7.73,6.84,6.10
7,0.523288,46.90,39.25,35.60,32.00,30.70,29.30,29.41,27.18,26.60,...,17.40,15.55,13.90,12.85,12.25,11.10,9.66,8.85,7.85,7.00
8,0.619178,48.38,41.00,38.00,34.15,33.05,29.00,31.05,29.28,28.24,...,19.25,17.05,15.74,14.98,14.45,13.00,11.90,10.75,9.25,8.50
9,0.695890,49.08,42.08,38.64,35.43,34.33,32.75,31.65,31.15,29.90,...,20.87,18.37,17.37,16.52,16.00,14.50,12.57,11.68,10.64,9.93


In [ ]:
def implied_vol(S0, K, T, market_price,flag, r=0.04, tol=0.00001):
    """Compute the implied volatility of a European Option
        S0: stock price
        K:  strike price
        T:  time to maturity
        r:  risk-free rate
        market_price: option price
        tol: user choosen tolerance
    """
    max_iter = 500 #max number of iterations
    vol_old = 0.30 #initial guess
    for k in range(max_iter):
        bs_price = bs(flag, S0, K, T, r, vol_old)
        Cprime =  vega(flag, S0, K, T, r, vol_old)*100
        C = bs_price - market_price
        vol_new = vol_old - C/Cprime
        bs_new = bs(flag, S0, K, T, r, vol_new)
        if (abs(vol_old - vol_new) < tol or abs(bs_new - market_price) < tol):
            break
        vol_old = vol_new

    implied_vol = vol_old
    return implied_vol*100

In [ ]:
strikes_call = list(call_data.columns)
strikes_put = list(put_data.columns)
strikes = [value for value in strikes_call if value in strikes_put]
if 'TTM' in strikes:
    strikes.remove('TTM')

iv_call = call_data.copy()
iv_put = put_data.copy()
iv_call = iv_call[strikes]
iv_put = iv_put[strikes]

for strike in strikes:
  for i in range(len(call_data)):
    iv_call.loc[i, strike] = implied_vol(spot_price, strike, call_data.loc[i, 'TTM'],call_data.loc[i, strike], 'c')
    iv_put.loc[i, strike] = implied_vol(spot_price, strike, put_data.loc[i, 'TTM'],put_data.loc[i, strike], 'p')

iv_call = iv_call.dropna(axis=1, how='any')
iv_put = iv_put.dropna(axis=1, how='any')
strikes_call = list(iv_call.columns)
strikes_put = list(iv_put.columns)
strikes = [value for value in strikes_call if value in strikes_put]
if 'TTM' in strikes:
    strikes.remove('TTM')
iv_call = iv_call[strikes]
iv_put = iv_put[strikes]

#print(strikes)
iv_call

strike,110.0,112.0,114.0,115.0,118.0,120.0,122.0,124.0,125.0,126.0,128.0,130.0,132.0,134.0,135.0,140.0,150.0
0,53.706444,42.207276,47.860493,46.158808,50.826152,49.058525,49.686266,49.793951,47.037388,47.140948,49.364103,49.115955,49.526120,49.265973,49.378333,48.713981,48.978371
1,49.713800,48.306723,48.195599,48.173547,50.987253,50.402724,50.700101,50.155665,49.883237,50.012166,49.893583,49.816379,49.952431,49.771545,50.007996,49.657109,49.902925
2,56.191574,73.947736,57.189261,57.097707,50.017645,58.125708,59.036665,60.132512,58.702383,57.920434,58.285778,58.159611,58.537952,58.327100,58.343586,57.616724,58.474593
3,55.614759,51.905934,51.743348,55.970095,56.149746,56.221822,55.438843,55.070729,55.607514,55.666893,55.891764,55.525662,55.112291,54.945850,55.463350,55.151769,55.180483
4,52.509234,52.470799,54.197893,51.642484,53.160609,53.852203,54.283591,53.380523,52.334241,53.538177,52.852137,53.470714,53.264700,53.548670,53.631594,53.208872,53.071004
5,53.621693,61.135309,46.106873,52.534498,49.292457,52.841360,49.303171,53.077472,52.754484,53.566575,52.922518,53.078602,53.175443,51.930619,52.501632,53.070210,52.630063
6,54.970128,53.552576,52.905116,53.116342,52.912817,53.880396,53.594359,52.593462,54.307697,52.595969,51.744820,52.615531,53.323325,52.141620,53.200408,53.052576,52.964501
7,51.847192,51.579796,50.899007,53.088621,51.396375,52.895074,52.809107,53.123282,53.118394,51.531573,50.811841,52.662140,52.424706,52.502438,52.011972,52.343020,52.011080
8,53.537316,53.727281,45.124652,52.819856,52.420000,52.442278,52.781838,54.005422,51.031226,50.363720,52.631787,52.756424,52.598093,52.659073,51.459141,52.065566,51.857679
9,53.630379,53.730976,52.427984,50.924556,53.859052,53.258761,53.756400,52.293494,52.887500,53.456164,52.711559,52.680681,51.006977,52.193041,50.956843,52.424434,52.316951


In [ ]:
var_call = iv_call.applymap(lambda x: (x/100)**2)
var_put = iv_put.applymap(lambda x: (x/100)**2)
var_call['TTM'] = (call_data['TTM']*365).astype(int)
var_put['TTM'] = (put_data['TTM']*365).astype(int)
var_call

strike,110.0,112.0,114.0,115.0,118.0,120.0,122.0,124.0,125.0,126.0,128.0,130.0,132.0,134.0,135.0,140.0,150.0,TTM
0,0.288438,0.178145,0.229063,0.213064,0.258330,0.240674,0.246873,0.247944,0.221252,0.222227,0.243681,0.241238,0.245284,0.242714,0.243822,0.237305,0.239888,30
1,0.247146,0.233354,0.232282,0.232069,0.259970,0.254043,0.257050,0.251559,0.248834,0.250122,0.248937,0.248167,0.249525,0.247721,0.250080,0.246583,0.249030,37
2,0.315749,0.546827,0.327061,0.326015,0.250176,0.337860,0.348533,0.361592,0.344597,0.335478,0.339723,0.338254,0.342669,0.340205,0.340397,0.331969,0.341928,44
3,0.309300,0.269423,0.267737,0.313265,0.315279,0.316089,0.307347,0.303279,0.309220,0.309880,0.312389,0.308310,0.303736,0.301905,0.307618,0.304172,0.304489,72
4,0.275722,0.275318,0.293741,0.266695,0.282605,0.290006,0.294671,0.284948,0.273887,0.286634,0.279335,0.285912,0.283713,0.286746,0.287635,0.283118,0.281653,100
5,0.287529,0.373753,0.212584,0.275987,0.242975,0.279221,0.243080,0.281722,0.278304,0.286938,0.280079,0.281734,0.282763,0.269679,0.275642,0.281645,0.276992,128
6,0.302171,0.286788,0.279895,0.282135,0.279977,0.290310,0.287236,0.276607,0.294933,0.276634,0.267753,0.276839,0.284338,0.271875,0.283028,0.281458,0.280524,163
7,0.268813,0.266048,0.259071,0.281840,0.264159,0.279789,0.278880,0.282208,0.282156,0.265550,0.258184,0.277330,0.274835,0.275651,0.270525,0.273979,0.270515,190
8,0.286624,0.288662,0.203623,0.278994,0.274786,0.275019,0.278592,0.291659,0.260419,0.253650,0.277010,0.278324,0.276656,0.277298,0.264804,0.271082,0.268922,226
9,0.287622,0.288702,0.274869,0.259331,0.290080,0.283650,0.288975,0.273461,0.279709,0.285756,0.277851,0.277525,0.260171,0.272411,0.259660,0.274832,0.273706,254


In [ ]:
for strike in strikes:
  for i in range(len(var_call)-1):
    for j in range(i+1, len(var_call)):
      near_var = var_call.loc[i, strike]
      near_expiry = var_call.loc[i, 'TTM']
      far_var = var_call.loc[j, strike]
      far_expiry = var_call.loc[j, 'TTM']

      annual_forward_variance = near_var - far_var
      if (annual_forward_variance > 0):
        forward_volatility = np.sqrt(annual_forward_variance)*100
        near_iv = iv_call.loc[i, strike]
        #print(f'FV: {forward_volatility} and IV: {near_iv} CE')
        if (forward_volatility > near_iv):
          print(f'Go Long {strike} CE expiring in {near_expiry} days and Go Short {strike} CE expiring in {far_expiry} days')

for strike in strikes:
  for i in range(len(var_put)-1):
    for j in range(i+1, len(var_put)):
      near_var = var_put.loc[i, strike]
      near_expiry = var_put.loc[i, 'TTM']
      far_var = var_put.loc[j, strike]
      far_expiry = var_put.loc[j, 'TTM']

      annual_forward_variance = near_var - far_var
      if annual_forward_variance > 0:
        forward_volatility = np.sqrt(annual_forward_variance) * 100
        near_iv = iv_put.loc[i, strike]
        #print(f'FV: {forward_volatility} and IV: {near_iv} PE')
        if forward_volatility > near_iv:
          print(f'Go Long {strike} PE expiring in {near_expiry} days and Go Short {strike} PE expiring in {far_expiry} days')